<a href="https://colab.research.google.com/github/Tanmaey45/Pyomo-codes/blob/main/prob_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyomo
!apt-get install -y -qq glpk-utils

import pyomo.environ as pyo
from pyomo.opt import SolverFactory


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [ ]:
model = pyo.ConcreteModel()

#sets
model.i = pyo.RangeSet(1,4)  #cities
model.j = pyo.RangeSet(1,3)   #plants

Price_data = { (1,1) : 8, (1,2) : 9 , (1,3) : 14,
         (2,1) : 6 , (2,2) : 12 , (2,3): 9,
         (3,1) : 10, (3,2) : 13, (3,3) : 16,
         (4,1) : 9, (4,2) : 7, (4,3): 5}

dmd_data = {1: 45,2 : 20, 3 :30 , 4 :30}
Capa_data = {1 :35, 2 :50, 3 :40}

#parameters
#Pi are ith power plants
model.P = pyo.Param(model.i, model.j,  initialize = Price_data)
P = model.P

model.dmd = pyo.Param(model.i, initialize = dmd_data)
dmd = model.dmd

model.Capa = pyo.Param(model.j, initialize = Capa_data)
Capa = model.Capa


In [ ]:
model.x = pyo.Var(model.i, model.j, within = pyo.NonNegativeReals)
x = model.x

#obj function
def obj_rule(model):
  return sum(sum(P[i,j]*x[i,j] for j in model.j) for i in model.i)

model.Obj = pyo.Objective(rule = obj_rule, sense = pyo.minimize)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
#constraint

def cond1(model,i):
  return sum(x[1,j] for j in model.j) >= dmd[1]

def cond2(model,i):
  return sum(x[2,j] for j in model.j) >= dmd[2]

def cond3(model,i):
  return sum(x[3,j] for j in model.j) >= dmd[3]

def cond4(model,i):
  return sum(x[4,j] for j in model.j) >= dmd[4]

def conc1(model,j):
  return sum(x[i,1] for i in model.i) <= Capa[1]

def conc2(model,j):
  return sum(x[i,2] for i in model.i) <= Capa[2]

def conc3(model,j):
  return sum(x[i,3] for i in model.i) <= Capa[3]

model.d1 = pyo.Constraint(model.i, rule = cond1)
model.d2 = pyo.Constraint(model.i, rule = cond2)
model.d3 = pyo.Constraint(model.i, rule = cond3)
model.d4 = pyo.Constraint(model.i, rule = cond4)
model.c1 = pyo.Constraint(model.j, rule = conc1)
model.c2 = pyo.Constraint(model.j, rule = conc2)
model.c3 = pyo.Constraint(model.j, rule = conc3)

In [ ]:
z = SolverFactory('glpk')
results = z.solve(model)

print(results)
print(model.Obj())

for j in model.j:
  for i in model.i:
    print("Q",j,i,"=", x[i,j]())




Problem: 
- Name: unknown
  Lower bound: 1020.0
  Upper bound: 1020.0
  Number of objectives: 1
  Number of constraints: 25
  Number of variables: 12
  Number of nonzeros: 84
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0040607452392578125
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

1020.0
Q 1 1 = 0.0
Q 1 2 = 10.0
Q 1 3 = 25.0
Q 1 4 = 0.0
Q 2 1 = 45.0
Q 2 2 = 0.0
Q 2 3 = 5.0
Q 2 4 = 0.0
Q 3 1 = 0.0
Q 3 2 = 10.0
Q 3 3 = 0.0
Q 3 4 = 30.0
